In [2]:
import random
import numpy as np

## 1A: Initializing the state space

In [4]:
class State:
    
    def __init__(self, x_position, y_position, heading):
        self.x_position = x_position
        self.y_position = y_position
        self.heading = heading
        
    def return_current_state(self):
        return self.x_position, self.y_position, self.heading

## 1B: Initializing the action space

In [5]:
class Action:
    
    def __init__(self, move, rotation):
        self.move = move
        self.rotation = rotation
        
    def return_action(self):
        return self.move, self.rotation

## 1C: Return the probabilty Psa(s') given pe, s, a, s'

### This is the Markov decision process

In [1]:
class MarkovDecisionProcess:
    
    # get the length and the width of the space that we are in
    def __init__(self, length, width):
        self.length = length
        self.width = width
    
    # check for bound violations
    def check_bounds(next_x, next_y):
        if (next_x < 0 or next_x >= self.width):
            next_x = current_x
        if (next_y < 0 or next_y >= self.height):
            next_y = current_y
        return next_x, next_y
    
    #define the directions
    left = {8,9,10}
    right = {2,3,4}
    up = {11,0,1}
    down = {5,6,7}
    
    # find out the next step --> returns next_x, next_y, next_heading
    def next_state_compiled(self, state, action):
        # get x, y and heading from state
        current_x = state.x_position
        current_y = state.y_position
        current_heading = state.heading
        
        # get move and rotation from action
        move = action.move
        rotation = action.rotation
        
        # check if heading is left, need to only change x and not y
        if heading in self.left:
            next_x = current_x - move
            next_y = current_y
        
        # check if heading is right, need to only change x and not y
        elif heading in self.right:
            next_x = current_x + move
            next_y = current_y
        
        # check if heading is up, need to only change y and not x
        elif heading in self.up:
            next_x = current_x
            next_y = current_y + move
        
        # check if heading is down, need to only change y and not x
        else:
            next_x = current_x
            next_y = current_y - move
        
        # need to check the bounds
        next_x, next_y = check_bounds(next_x, next_y)
        
        # finally, need to accomodate for the rotation in heading
        next_heading = (current_heading + rotation) % 12
        
        return next_x, next_y, next_heading
    
    # find out the next step for the movements that have the prob of error
    def next_state_individualized(self, current_x, current_y, current_heading, move, rotation):
        # check if heading is left, need to only change x and not y
        if heading in self.left:
            next_x = current_x - move
            next_y = current_y
        
        # check if heading is right, need to only change x and not y
        elif heading in self.right:
            next_x = current_x + move
            next_y = current_y
        
        # check if heading is up, need to only change y and not x
        elif heading in self.up:
            next_x = current_x
            next_y = current_y + move
        
        # check if heading is down, need to only change y and not x
        else:
            next_x = current_x
            next_y = current_y - move
        
        # need to check the bounds
        next_x, next_y = check_bounds(next_x, next_y)
        
        # finally, need to accomodate for the rotation in heading
        next_heading = (current_heading + rotation) % 12
        
        return next_x, next_y, next_heading
    
    # find out the transition probabilities
    def transition_probabilities(self, prob_of_error, action, current_state, next_state):
        # get move and rotation from action
        move = action.move
        rotation = action.rotation
        
        # if we do not move, current state should equal next state. 
        if move == 0:
            if current_state.return_current_state == next_state.return_current_state:
                return 1
            else:
                return 0
        
        # in the case that we do move, need to account for the error probabilities
        if next_state.return_current_state == self.next_state_compiled(current_state, action):
            return 1-(2*prob_of_error)
        else:
            current_x = current_state.x_position
            current_y = current_state.y_position
            current_heading = current_state.heading

            # need to check to return the prob of error
            if next_state.return_current_state == self.next_state_individualized(current_x, current_y, (current_heading-1)%12, move, rotation):
                return prob_of_error

            if next_state.return_current_state == self.next_state_individualized(current_x, current_y, (current_heading+1)%12, move, rotation):
                return prob_of_error
        
        return 0
    
    number_of_headings = 12
    
    # Part 1D
    def compute_next_state(self, prob_of_error, current_state, action):
        
        wrong_states = []
        # need to loop through all of the values of x, y and heading
        for i in range(self.length):
            for j in range(self.width):
                for k in range(self.number_of_headings):
                    # define the next state
                    next_state = State(i,j,k)
                    # find the transition probability between current state and the next state
                    prob_of_transition = self.transition_probabilities(prob_of_error, action, current_state, next_state)
                    # if the probability of transitioning does not equal 0, then only do we proceed. If 0, do not care
                    if prob_of_transition != 0:
                        # if our transition probabilty is the probability of error, it means that there was an error. Need to keep track of these next state values
                        if prob_of_transition == prob_of_error:
                            wrong_states.append(next_state)
                        else:
                            correct_next_state = next_state
        # if I choose a random number between 0 and 1 and my number is less than 2*prob_of_error, I move in the wrong direction. If not, I move in the correct direction.
        random_number_generated = np.random.uniform(0,1)
        if random_number_generated < 2*prob_of_error:
            random_index = random.randrange(len(wrong_states))
            return wrong_states[random_index]
        else:
            return correct_next_state  

## Question 2: Write a function that returns the reward R(s) given input s.

In [8]:
def reward(self, current_state):
    # define the lengh and the width
    length = width = 6
    
    # get the current x, y positions -- do not care about heading
    current_x, current_y, _ = current_state.return_current_state
    
    # define the rewards
    if current_x == 3 and current_y == 4:
        reward = 1
    if current_x == 0 or current_x == width-1:
        reward = -100
    if current_y == 0 or current_y == height-1:
        reward = -100
    if current_x == 2 or current_x == 4:
        if current_y == 2 or current_y == 3 or current_y == 4:
            reward = -1
    return reward